In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect


NOTE: User is responsible for checking the content of datasets and the applicable licenses and determining if suitable for the intended use.
"""
# If you're using Google Colab and not running locally, run this cell.
import os

# # Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]


# Automatic Speech Recognition with Transducer Models using HF Datasets

We have discussed training various ASR models in NeMo using custom datasets, either for fine-tuning or for scratch-training. In this tutorial, we will showcase how to use Hugging Face datasets library in order to finetune a Transducer ASR model on a small dataset from for the Telugu language. 
It includes discussions relevant to preparing datasets with HF and how to use them to finetune NeMo models. The same method applies to training from scratch. However, for training, we recommend using scripts directly from the `examples/asr/` folder.

In this tutorial, we demonstrate the usage of HF datasets for the Telugu language, where we use the Fluers dataset for training, validation, and testing. However, the same procedure can be used for other languages or domains and finetuned for specific use cases accordingly. 

For scripts, refer to [speech_to_text_finetune.py]('https://github.com/NVIDIA/NeMo/blob/main/examples/asr/speech_to_text_finetune.py') for training from scratch. 

--------

**Note**: It is assumed that the previous tutorial - "Intro-to-Transducers" has been reviewed, and there is some familiarity with the config components of transducer models.

# Preparing the dataset

In this tutorial, we will be utilizing the `Telugu` language dataset part of fluers set. More details about dataset can be found at: https://huggingface.co/datasets/google/fleurs/viewer/te_in 

Dataset consists of 3 splits: `train`, `validation`, and `test`. We will use the `train` and `validation` splits for training and validation, respectively. The `test` split will be used for testing the model. Train set consists of 2,300 utterances and validation set consists of 311 examples. 

You may listen to some of the samples from the dataset using the following code:

```python

```

In [ ]:
import datasets
import IPython.display as ipd
# sample from the validation set
sample = datasets.load_dataset('google/fleurs', 'te_in', split='validation')[0]
 
# Let's listen to the audio file and its transcription
ipd.Audio(sample['audio']['array'], rate=16000)


Lets look at the transcription


In [ ]:
print("Transcription:", sample['transcription'])

Audio samples are in the `audio` field and the corresponding transcriptions are in the `transcription` field. We will use these fields to prepare data and train our model.


# Preparing tokenizer

Since we are finetuning Parakeet model, which is an English language model, we need to update the tokenizer and update the decoder to support the new language. 

First, we will extract text transcriptions from the dataset and use them to train a tokenizer. We will use the scripts from NeMo first to get the data from HF dataset using `get_hf_dataset.py` script. Next we use `process_asr_text_tokenizer.py` script to prepare the tokenizer from [scripts](https://github.com/NVIDIA/NeMo/tree/main/scripts/tokenizers) folder. 


Download the `get_hf_text_data.py` script from the [scripts](https://github.com/NVIDIA/NeMo/blob/main/scripts/tokenizers) folder and run the following command to get the data from HF dataset. 

In [ ]:
if not os.path.exists("scripts/get_hf_text_data.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tokenizers/get_hf_text_data.py

Major difference from NeMo dataset configs to training using HF-datasets is for using hf-datasets, users need to mention the hf-dataset information through hf data config and pass to the script for downloading necessary data. Users can switch to another dataset by changing necessary fields in the hf data config. 
Let's create that config here.

In [ ]:
from omegaconf import OmegaConf, open_dict
train_split = {
            'path': 'google/fleurs',
            'name': 'te_in',
            'split': 'train', # we will update this accordingly based on the split
            'streaming': False}
print(OmegaConf.to_yaml(train_split))

Since we need clean data for training tokenizer and models, we need to filter the data based on how dataset was constructed and how we would like the ASR model output to be. 

Based on prior analysis of text transcripts of the current hf dataset, we skip all non-alphanumeric characters except `full-stop` using `normalize_text` option of the `get_hf_text_data.py` script, based on `huggingface_data_tokenizer.yaml` config file. 

In [ ]:

if not os.path.exists('configs/huggingface_data_tokenizer.yaml'):
    !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tokenizers/conf/huggingface_data_tokenizer.yaml


!export HYDRA_FULL_ERROR=1;python scripts/get_hf_text_data.py \
    --config-path="../configs" \
    --config-name="huggingface_data_tokenizer" \
    normalize_text=True \
    symbols_to_keep=["."] \
    text_key="transcription" \
    output_file='telugu_train_corpus.txt' \
    +hf_data_cfg='[{train_split}]'

From the above command, we were able to use the `huggingface_data_tokenizer.yaml` config file to download the data from HF dataset. The download data is saved to `telugu_train_corpus.txt` file, which we will use to train the tokenizer. Before that, let's look at some utterances from the normalized (preprocessed) text file.

In [ ]:
with open('telugu_train_corpus.txt', 'r') as f:
    for i in range(5):
        print(f.readline().strip())

As can be seen, there are characters from English and also numerical characters as well. For this tutorial we will retain those characters and prepare the tokenizer. But if you would like to filter out those characters, you can use your own function in the `get_hf_text_data.py` script to filter out the characters. 

Now, we will use the `process_asr_text_tokenizer.py` script to prepare the tokenizer, for finetuning we can limit the vocab size to 256 for telugu language.

In [ ]:

if not os.path.exists("scripts/process_asr_text_tokenizer.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/tokenizers/process_asr_text_tokenizer.py

# Now this downloads the text corpus of data to tokenizers script
VOCAB_SIZE = 256  # can be any value above 29
TOKENIZER_TYPE = "spe"  # can be wpe or spe
SPE_TYPE = "bpe"  # can be bpe or unigram

# ------------------------------------------------------------------- #
!rm -r tokenizers/

if not os.path.exists("tokenizers"):
  os.makedirs("tokenizers")

!python scripts/process_asr_text_tokenizer.py \
    --data_file='telugu_train_corpus.txt' \
    --data_root="tokenizers" \
    --tokenizer=$TOKENIZER_TYPE \
    --spe_type=$SPE_TYPE \
    --no_lower_case \
    --log \
    --vocab_size=$VOCAB_SIZE

In [ ]:
# Tokenizer path
if TOKENIZER_TYPE == 'spe':
  TOKENIZER = os.path.join("tokenizers", f"tokenizer_spe_{SPE_TYPE}_v{VOCAB_SIZE}")
  TOKENIZER_TYPE_CFG = "bpe"
else:
  TOKENIZER = os.path.join("tokenizers", f"tokenizer_wpe_v{VOCAB_SIZE}")
  TOKENIZER_TYPE_CFG = "wpe"

In [ ]:
print("Tokenizers are saved at:", TOKENIZER)

# Preparing a Transducer Model

Now that we have the dataset and tokenizer prepared, let us begin by setting up the config of the Transducer model! In this tutorial, we will finetune [Parakeet RNNT 0.6B](https://huggingface.co/nvidia/parakeet-rnnt-0.6b) model, but the same procedure can be used for other RNNT models as well.


## Prepare the config 
For finetuning the model, we need to update the config file to include the tokenizer and the dataset information. We will use the `parakeet-rnnt-6b` model and update the config file to include the tokenizer and the dataset information. For this we use `speech_to_text_hf_finetune.yaml` config file, and training script `speech_to_text_finetune.py` from the `examples/asr` folder.

For this demo, we will replicate only a portion of the script - in order to show just the necessary components for training the model on single GPU. However, we recommend users to use the scripts directly from the `examples/asr` folder for training the model on multiple GPUs.

In [ ]:
## Grab the config we'll use in this example
!mkdir -p configs
if not os.path.exists('configs/speech_to_text_hf_finetune.yaml'):
    !wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/asr_finetune/speech_to_text_hf_finetune.yaml

config = OmegaConf.load("configs/speech_to_text_hf_finetune.yaml")

Update tokenizer info

In [ ]:
config.model.tokenizer.update_tokenizer = True
config.model.tokenizer.dir = TOKENIZER
config.model.tokenizer.type = TOKENIZER_TYPE_CFG
print(OmegaConf.to_yaml(config.model.tokenizer))

Need to mention the same data preprocessing scripts here as HF datasets are processed batchwise and the same preprocessing scripts are used for applying the same preprocessing to the dataset, as was used for training the tokenizer. 

In [ ]:

config.model.train_ds.normalize_text=False
config.model.normalize_text = True # same as the normalize_text in the get_hf_text_data.py
config.model.symbols_to_keep=["."] # same as the symbols_to_keep in the get_hf_text_data.py
config.model.data_path = 'google/fleurs'
config.model.data_name = 'te_in'
config.model.streaming = False


Setting up dataset config for validation and test datasets is similar to the training dataset.

In [ ]:
config.model.train_ds.hf_data_cfg=train_split
config.model.train_ds.text_key='transcription'
config.model.train_ds.batch_size=16 # change this based on your GPU memory.
config.model.train_ds.normalize_text=True

config.model.validation_ds.hf_data_cfg=train_split
config.model.validation_ds.hf_data_cfg.split='validation' # updated this based on the split
config.model.validation_ds.text_key='transcription'
config.model.validation_ds.normalize_text=True

config.model.test_ds.hf_data_cfg=train_split
config.model.test_ds.hf_data_cfg.split='test' # updated this based on the split
config.model.test_ds.text_key='transcription'
config.model.test_ds.normalize_text=True

print(OmegaConf.to_yaml(config.model))

Update the trainer config to include the necessary parameters for training the model.

## Training the model

For finetuning the model, basic parameters we need to primarily focus on are optimizer, scheduler, learning rate and the number of steps to finetune.

For this example we will stick to AdamW optimizer and CosineAnnealing scheduler, for 2500 steps. 
These parameters can be updated based on the dataset and the model being used.

In [ ]:
print(OmegaConf.to_yaml(config.model.optim))

In [ ]:
# Current optim looks fine except for the warmup_steps
config.model.optim.sched.warmup_steps = 500 # 10% of the total steps
config.model.optim.lr = 3e-4

del config.model.spec_augment #For this example, we are not using SpecAugment

Let's load the model and also the dataloader for training the model

In [ ]:
from nemo.collections.asr.models import ASRModel
asr_model=ASRModel.from_pretrained('nvidia/parakeet-rnnt-0.6b')

After loading the pretrained model, decoder of the model needs to updated to support the new language.

In [ ]:

decoder=asr_model.decoder.state_dict()
joint_state = asr_model.joint.state_dict()
prev_vocab_size = asr_model.tokenizer.vocab_size
asr_model.change_vocabulary(new_tokenizer_dir=TOKENIZER, new_tokenizer_type=TOKENIZER_TYPE_CFG)
if asr_model.tokenizer.vocab_size == prev_vocab_size: # checking new tokenizer vocab size
    asr_model.decoder.load_state_dict(decoder)
    asr_model.joint.load_state_dict(joint_state)

Loading dataloaders

In [ ]:
from nemo.utils import logging, model_utils
cfg = model_utils.convert_model_config_to_dict_config(config)
asr_model.setup_training_data(cfg.model.train_ds)
asr_model.setup_validation_data(cfg.model.validation_ds)
if hasattr(cfg.model, 'test_ds') and cfg.model.test_ds.manifest_filepath is not None:
    asr_model.setup_test_data(cfg.model.test_ds)

In [ ]:
# We will also reduce the hidden dimension of the joint and the prediction networks to preserve some memory
asr_model.setup_optimization(cfg.model.optim)

------

Setup a Pytorch Lightning Trainer:

We can reduce the number of steps to 5000 for this small dataset.
and also update precision to float16 for faster training. # Change this bf16 training on Ampere based GPUs. 

In [ ]:
import torch
from pytorch_lightning import Trainer

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'gpu'

MAX_STEPS = 5000

# Initialize a Trainer for the Transducer model
trainer = Trainer(devices=1, accelerator=accelerator, max_epochs=-1, max_steps=MAX_STEPS,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=100, check_val_every_n_epoch=10, precision='bf16')

In [ ]:
# Build the model
trainer.fit(asr_model)

Once trained users can save the model and use it for inference.

In [ ]:
asr_model.save_to("telugu_asr_model.nemo")

Running inference on  a sample from the test dataset

In [ ]:
del asr_model
asr_model = ASRModel.restore_from("telugu_asr_model.nemo")

!rm -r telugu_asr_model.nemo # remove the model to save space

# Let's test the model on a sample from the test set
sample = datasets.load_dataset('google/fleurs', 'te_in', split='test')[0]
transcription = sample['transcription']

# Let's listen to the audio file and its transcription
ipd.Audio(sample['audio']['array'], rate=16000)

In [ ]:
asr_model.eval()
# save audio file to disk
audio_path = "sample.wav"
import soundfile as sf
sf.write(audio_path, sample['audio']['array'], 16000)
transcription = asr_model.transcribe([audio_path])

In [ ]:

print("Predicted:", transcription[0])
print("Target:", sample['transcription'])

In [ ]:
os.remove(audio_path) # remove the audio file to save space